# **OVERVIEW OF THE DATASET**

- Definition of sleep event: 
    - Event class: Onset/Wakeup
    - Sleep Onset is indicate the beginning of the sleep, and wakeup is defined as the ebd of the sleep.
    - Single sleep period must be the longest inactive period while the accelerometer is worn. 
    - Single sleep must be the at leat 30 minutes in length which can be interrupted by bouts that do not exceed 30 consecutive 30 mm,inutes.
    
    
- The individual can have multiple sleep window in same clander day. But the **longest sleep window during the night will be recored**.


- There is no strict rule to limit number of sleep events in a givien period **however only one window assign per nigth**.


- The event that annotated when the accelerometer removed should be refrain from makking predictions. 

- **Target variable: Event (Oneset/Wakeup)**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
import polars as pl
import datetime as dt
import plotly.express as px

- Since the dataset is quite big, Used polars to load fast and effectively in memory.
- Converted the outpiut into pandas dataframe to perform EDA and analysis.

In [2]:
# reading train_series parquet file as well as converting the timestamp to datetime dtype
train_series = (pl.scan_parquet("/kaggle/input/child-mind-institute-detect-sleep-states/train_series.parquet")
    .with_columns((pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z")),
                  ((pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z")).dt.year().alias('year')),
                 ((pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z")).dt.month().alias('month')),
                 ((pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z")).dt.day().alias('day')),
                 ((pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z")).dt.hour().alias('hour')))
    .collect().to_pandas())

# reading train_event csv
train_event = (pl.scan_csv("/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv")
              .with_columns((pl.col('timestamp').str.strptime(pl.Datetime,"%Y-%m-%dT%H:%M:%S%Z")),
                           ((pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z")).dt.year().alias('year')),
                           ((pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z")).dt.month().alias('month')),
                           ((pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z")).dt.day().alias('day')),
                           ((pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S%Z")).dt.hour().alias('hour')))
              .collect().to_pandas())

In [3]:
# understanding the features 
train_series.info()
print(f"\n The shape of the Dataset is : \n rows = {train_series.shape[0]}, columns = {train_series.shape[1]}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127946340 entries, 0 to 127946339
Data columns (total 9 columns):
 #   Column     Dtype         
---  ------     -----         
 0   series_id  object        
 1   step       uint32        
 2   timestamp  datetime64[ns]
 3   anglez     float32       
 4   enmo       float32       
 5   year       int32         
 6   month      uint32        
 7   day        uint32        
 8   hour       uint32        
dtypes: datetime64[ns](1), float32(2), int32(1), object(1), uint32(4)
memory usage: 5.2+ GB

 The shape of the Dataset is : 
 rows = 127946340, columns = 9


In [4]:
# understanding the features 
train_event.info()
print(f"\n The shape of the Dataset is : \n rows = {train_event.shape[0]}, columns = {train_event.shape[1]}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14508 entries, 0 to 14507
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   series_id  14508 non-null  object        
 1   night      14508 non-null  int64         
 2   event      14508 non-null  object        
 3   step       9585 non-null   float64       
 4   timestamp  9585 non-null   datetime64[ns]
 5   year       9585 non-null   float64       
 6   month      9585 non-null   float64       
 7   day        9585 non-null   float64       
 8   hour       9585 non-null   float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(2)
memory usage: 1020.2+ KB

 The shape of the Dataset is : 
 rows = 14508, columns = 9


# Getting insight from Train_series

In [5]:
# display the top 5 rows
train_series.head()

,series_id,step,timestamp,anglez,enmo,year,month,day,hour
0,038441c925bb,0,2018-08-14 15:30:00,2.6367,0.0217,2018,8,14,15
1,038441c925bb,1,2018-08-14 15:30:05,2.6368,0.0215,2018,8,14,15
2,038441c925bb,2,2018-08-14 15:30:10,2.6370,0.0216,2018,8,14,15
3,038441c925bb,3,2018-08-14 15:30:15,2.6368,0.0213,2018,8,14,15
4,038441c925bb,4,2018-08-14 15:30:20,2.6368,0.0215,2018,8,14,15


In [6]:
# Five point summary
train_series.describe(include='all').T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
series_id,127946340,277,78569a801a38,1433880,NaN,NaN,NaN,NaN,NaN,NaN,NaN
step,127946340.0,NaN,NaN,NaN,254804.797188,0.0,115812.0,234519.0,357196.0,1433879.0,177892.965593
timestamp,127946340,NaN,NaN,NaN,2018-07-16 13:31:51.766071296,2017-08-04 17:30:00,2018-02-12 20:09:30,2018-07-22 02:40:47.500000,2018-12-24 18:28:05,2019-07-09 09:44:55,NaN
anglez,127946340.0,NaN,NaN,NaN,-8.810453,-90.0,-31.8589,-9.5979,11.3002,90.0,35.521877
enmo,127946340.0,NaN,NaN,NaN,0.041315,0.0,0.0013,0.0172,0.0437,11.4337,0.101829
year,127946340.0,NaN,NaN,NaN,2018.039475,2017.0,2018.0,2018.0,2018.0,2019.0,0.650516
month,127946340.0,NaN,NaN,NaN,6.504757,1.0,3.0,6.0,10.0,12.0,3.664508
day,127946340.0,NaN,NaN,NaN,15.89134,1.0,8.0,16.0,23.0,31.0,8.783455
hour,127946340.0,NaN,NaN,NaN,11.520186,0.0,5.0,12.0,18.0,23.0,6.932357


In [7]:
# null value check
train_series.isnull().sum()

series_id    0
step         0
timestamp    0
anglez       0
enmo         0
year         0
month        0
day          0
hour         0
dtype: int64

# Initial observation - Train_series
- There are 277 accelerometer series_id.
- The step is unique identifier of the observation.
- The anglez value range from -90 to +90 which is **inline with the angle of the arm relative to the vertical axis of the body.** (complete 180 degree angle)
- There are no null values in the train_series.

# Getting insight from Train_event

In [8]:
# display tpo 5 rows
train_event.head()

,series_id,night,event,step,timestamp,year,month,day,hour
0,038441c925bb,1,onset,4992.0,2018-08-14 22:26:00,2018.0,8.0,14.0,22.0
1,038441c925bb,1,wakeup,10932.0,2018-08-15 06:41:00,2018.0,8.0,15.0,6.0
2,038441c925bb,2,onset,20244.0,2018-08-15 19:37:00,2018.0,8.0,15.0,19.0
3,038441c925bb,2,wakeup,27492.0,2018-08-16 05:41:00,2018.0,8.0,16.0,5.0
4,038441c925bb,3,onset,39996.0,2018-08-16 23:03:00,2018.0,8.0,16.0,23.0


In [9]:
# Five point summary
train_event.describe(include='all').T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
series_id,14508,277,78569a801a38,168,NaN,NaN,NaN,NaN,NaN,NaN,NaN
night,14508.0,NaN,NaN,NaN,15.120072,1.0,7.0,14.0,21.0,84.0,10.286758
event,14508,2,onset,7254,NaN,NaN,NaN,NaN,NaN,NaN,NaN
step,9585.0,NaN,NaN,NaN,214352.123944,936.0,95436.0,200604.0,317520.0,739392.0,141268.408192
timestamp,9585,NaN,NaN,NaN,2018-07-01 05:31:32.450704128,2017-08-05 22:44:00,2018-01-25 22:09:00,2018-06-16 23:05:00,2018-12-17 06:36:00,2019-07-05 05:33:00,NaN
year,9585.0,NaN,NaN,NaN,2017.997809,2017.0,2018.0,2018.0,2018.0,2019.0,0.660645
month,9585.0,NaN,NaN,NaN,6.496609,1.0,3.0,6.0,10.0,12.0,3.66713
day,9585.0,NaN,NaN,NaN,15.987585,1.0,8.0,16.0,24.0,31.0,8.818433
hour,9585.0,NaN,NaN,NaN,12.127804,0.0,6.0,8.0,21.0,23.0,7.944063


In [10]:
# extracting the week name 
train_event['week'] = train_event['timestamp'].dt.day_name()

In [11]:
# check for null values
train_event.isnull().sum()

series_id       0
night           0
event           0
step         4923
timestamp    4923
year         4923
month        4923
day          4923
hour         4923
week         4923
dtype: int64

In [12]:
# Visualize the hour distribution with respective to event
px.histogram(data_frame=train_event,x='hour',color='event',nbins=15,marginal='box')

In [13]:
px.histogram(data_frame=train_event[~train_event.isnull().any(axis=1)],x='hour',color='event',nbins=5,facet_col='week',
            facet_col_wrap=2)

# Initial observation - Train Event

- Dataset consist of two years data (2017 to 2019)
- Major wakeup time at 6-8 am and onset time is at 20-23 pm at night which is inline which the human behaviour 

# Merge the dataset train event and train series

In [14]:
train_data = pd.merge(train_series,train_event,how='left',on=['series_id','timestamp'])
train_data.drop(columns=['year_y','month_y','day_y','hour_y','week'],inplace=True)
train_data['week'] = train_data['timestamp'].dt.day_name()

In [15]:
train_data.describe(include='all').T

,count,unique,top,freq,mean,min,25%,50%,75%,max,std
series_id,127946340,277,78569a801a38,1433880,NaN,NaN,NaN,NaN,NaN,NaN,NaN
step_x,127946340.0,NaN,NaN,NaN,254804.797188,0.0,115812.0,234519.0,357196.0,1433879.0,177892.965593
timestamp,127946340,NaN,NaN,NaN,2018-07-16 13:31:51.766071296,2017-08-04 17:30:00,2018-02-12 20:09:30,2018-07-22 02:40:47.500000,2018-12-24 18:28:05,2019-07-09 09:44:55,NaN
anglez,127946340.0,NaN,NaN,NaN,-8.810453,-90.0,-31.8589,-9.5979,11.3002,90.0,35.521877
enmo,127946340.0,NaN,NaN,NaN,0.041315,0.0,0.0013,0.0172,0.0437,11.4337,0.101829
year_x,127946340.0,NaN,NaN,NaN,2018.039475,2017.0,2018.0,2018.0,2018.0,2019.0,0.650516
month_x,127946340.0,NaN,NaN,NaN,6.504757,1.0,3.0,6.0,10.0,12.0,3.664508
day_x,127946340.0,NaN,NaN,NaN,15.89134,1.0,8.0,16.0,23.0,31.0,8.783455
hour_x,127946340.0,NaN,NaN,NaN,11.520186,0.0,5.0,12.0,18.0,23.0,6.932357
night,9588.0,NaN,NaN,NaN,12.861076,1.0,6.0,12.0,19.0,43.0,8.168208


In [16]:
train_data.head()

,series_id,step_x,timestamp,anglez,enmo,year_x,month_x,day_x,hour_x,night,event,step_y,week
0,038441c925bb,0,2018-08-14 15:30:00,2.6367,0.0217,2018,8,14,15,NaN,NaN,NaN,Tuesday
1,038441c925bb,1,2018-08-14 15:30:05,2.6368,0.0215,2018,8,14,15,NaN,NaN,NaN,Tuesday
2,038441c925bb,2,2018-08-14 15:30:10,2.6370,0.0216,2018,8,14,15,NaN,NaN,NaN,Tuesday
3,038441c925bb,3,2018-08-14 15:30:15,2.6368,0.0213,2018,8,14,15,NaN,NaN,NaN,Tuesday
4,038441c925bb,4,2018-08-14 15:30:20,2.6368,0.0215,2018,8,14,15,NaN,NaN,NaN,Tuesday
